Loading the required packages.

In [1]:
library(dplyr)
library(RSQLite)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Loading the sqlite database for european soccer.

In [2]:
my_db <- src_sqlite("database.sqlite")

1) Which are the top 10 players by overall rating?

Using sql query to extract the overall ratings of all players with their id and name.

In [3]:
Player<-tbl(my_db, sql("SELECT  Player.player_name,Player.player_api_id, MAX(overall_rating) AS overall_rating FROM Player_Attributes,Player WHERE Player.player_api_id=Player_Attributes.player_api_id GROUP BY Player.player_api_id"))

Checking the contents of Player table.

In [4]:
Player

Source:   query [?? x 3]
Database: sqlite 3.11.1 [Desktop/database.sqlite]

         player_name player_api_id overall_rating
               <chr>         <int>          <int>
1  Patryk Rachwal,18          2625             63
2        Diego Mainz          2752             72
3        Jose Dorado          2768             75
4   Ignacio Gonzalez          2770             74
5        Alberto Rey          2790             73
6     Javier Jimenez          2796             69
7    Pablo Hernandez          2802             82
8        Ruben Perez          2805             76
9         Ivan Perez          2857             72
10   Vicente Sanchez          2862             76
# ... with more rows

2) Which are the top 10 goalkeepers by sum of gk attributes?

Selecting the Gk attributes by taking the sum of all the gk columns and grouping them by player_api_id to pick the maximum value.

In [5]:
ques_2 <- tbl(my_db,sql("SELECT player_api_id,MAX(gk_diving+gk_handling+gk_kicking+gk_positioning+gk_reflexes) AS gk_attributes FROM Player_Attributes GROUP BY player_api_id"))
ques_2

Source:   query [?? x 2]
Database: sqlite 3.11.1 [Desktop/database.sqlite]

   player_api_id gk_attributes
           <int>         <int>
1           2625           153
2           2752            53
3           2768           128
4           2770           148
5           2790           144
6           2796           340
7           2802           146
8           2805           141
9           2857           143
10          2862           146
# ... with more rows

Joining the gk_attributes to the Player table using left join by the player_api_column.

In [6]:
Player <- Player %>% left_join(ques_2)

Joining, by = "player_api_id"


Now the Player table has the gk_attributes sorted according to the player_api_id.

In [7]:
Player

Source:   query [?? x 4]
Database: sqlite 3.11.1 [Desktop/database.sqlite]

         player_name player_api_id overall_rating gk_attributes
               <chr>         <int>          <int>         <int>
1  Patryk Rachwal,18          2625             63           153
2        Diego Mainz          2752             72            53
3        Jose Dorado          2768             75           128
4   Ignacio Gonzalez          2770             74           148
5        Alberto Rey          2790             73           144
6     Javier Jimenez          2796             69           340
7    Pablo Hernandez          2802             82           146
8        Ruben Perez          2805             76           141
9         Ivan Perez          2857             72           143
10   Vicente Sanchez          2862             76           146
# ... with more rows

3) Which are the top 10 players by number of appearances (with any team they played with)?

Calculating the number of appearances by each player by iterating all the columns through home_player_1 to home_player_10 and away_player_1 to away_player_10. [1]

In [8]:
ques_3<-tbl(my_db,sql("SELECT player_api_id, count(*) as num_appearances FROM (SELECT home_player_1 AS player_api_id FROM Match UNION ALL
                                 SELECT home_player_2 AS player_api_id FROM Match UNION ALL
                                 SELECT home_player_3 AS player_api_id FROM Match
                                 UNION ALL
                                 SELECT home_player_4 AS player_api_id FROM Match
                                 UNION ALL
                                 SELECT home_player_5 AS player_api_id FROM Match
                                 UNION ALL
                                 SELECT home_player_6 AS player_api_id FROM Match
                                 UNION ALL
                                 SELECT home_player_7 AS player_api_id FROM Match
                                 UNION ALL
                                 SELECT home_player_8 AS player_api_id FROM Match
                                 UNION ALL
                                 SELECT home_player_9 AS player_api_id FROM Match
                                 UNION ALL
                                 SELECT home_player_10 AS player_api_id FROM Match
                                 UNION ALL
                                 SELECT home_player_11 AS player_api_id FROM Match
                                 UNION ALL
                                 SELECT away_player_1 AS player_api_id FROM Match
                                 UNION ALL
                                 SELECT away_player_2 AS player_api_id FROM Match
                                 UNION ALL
                                 SELECT away_player_3 AS player_api_id FROM Match
                                 UNION ALL
                                 SELECT away_player_4 AS player_api_id FROM Match
                                 UNION ALL
                                 SELECT away_player_5 AS player_api_id FROM Match
                                 UNION ALL
                                 SELECT away_player_6 AS player_api_id FROM Match
                                 UNION ALL
                                 SELECT away_player_7 AS player_api_id FROM Match
                                 UNION ALL
                                 SELECT away_player_8 AS player_api_id FROM Match
                                 UNION ALL
                                 SELECT away_player_9 AS player_api_id FROM Match
                                 UNION ALL
                                 SELECT away_player_10 AS player_api_id FROM Match
                                 UNION ALL
                                 SELECT away_player_11 AS player_api_id FROM Match) GROUP BY player_api_id"))

Adding the number of appearances of each player to the player table using their player_api_id.

In [9]:
Player <- Player %>% inner_join(ques_3, by = "player_api_id")

Now the Player table has the number of appearances sorted according to the player_api_id.

In [10]:
Player

Source:   query [?? x 5]
Database: sqlite 3.11.1 [Desktop/database.sqlite]

         player_name player_api_id overall_rating gk_attributes num_appearances
               <chr>         <int>          <int>         <int>           <int>
1  Patryk Rachwal,18          2625             63           153              90
2        Diego Mainz          2752             72            53              81
3        Jose Dorado          2768             75           128              71
4   Ignacio Gonzalez          2770             74           148              16
5        Alberto Rey          2790             73           144              50
6     Javier Jimenez          2796             69           340               1
7    Pablo Hernandez          2802             82           146             144
8        Ruben Perez          2805             76           141             131
9         Ivan Perez          2857             72           143               3
10   Vicente Sanchez          2862          

4) Which are the top 10 players by number of leagues they played in?

Calculating the number of leagues played by each player by iterating all the columns through home_player_1 to home_player_10 and away_player_1 to away_player_10 and then grouping them by league_id.[1]

In [11]:
ques_4<-tbl(my_db,sql("SELECT player_api_id, count(distinct(league_id)) as num_leagues FROM (SELECT league_id,home_player_1 AS player_api_id FROM Match UNION ALL
                                 SELECT league_id,home_player_2 AS player_api_id FROM Match UNION ALL
                      SELECT league_id,home_player_3 AS player_api_id FROM Match
                      UNION ALL
                      SELECT league_id,home_player_4 AS player_api_id FROM Match
                      UNION ALL
                      SELECT league_id,home_player_5 AS player_api_id FROM Match
                      UNION ALL
                      SELECT league_id,home_player_6 AS player_api_id FROM Match
                      UNION ALL
                      SELECT league_id,home_player_7 AS player_api_id FROM Match
                      UNION ALL
                      SELECT league_id,home_player_8 AS player_api_id FROM Match
                      UNION ALL
                      SELECT league_id,home_player_9 AS player_api_id FROM Match
                      UNION ALL
                      SELECT league_id,home_player_10 AS player_api_id FROM Match
                      UNION ALL
                      SELECT league_id,home_player_11 AS player_api_id FROM Match
                      UNION ALL
                      SELECT league_id,away_player_1 AS player_api_id FROM Match
                      UNION ALL
                      SELECT league_id,away_player_2 AS player_api_id FROM Match
                      UNION ALL
                      SELECT league_id,away_player_3 AS player_api_id FROM Match
                      UNION ALL
                      SELECT league_id,away_player_4 AS player_api_id FROM Match
                      UNION ALL
                      SELECT league_id,away_player_5 AS player_api_id FROM Match
                      UNION ALL
                      SELECT league_id,away_player_6 AS player_api_id FROM Match
                      UNION ALL
                      SELECT league_id,away_player_7 AS player_api_id FROM Match
                      UNION ALL
                      SELECT league_id,away_player_8 AS player_api_id FROM Match
                      UNION ALL
                      SELECT league_id,away_player_9 AS player_api_id FROM Match
                      UNION ALL
                      SELECT league_id,away_player_10 AS player_api_id FROM Match
                      UNION ALL
                      SELECT league_id,away_player_11 AS player_api_id FROM Match) GROUP BY player_api_id ORDER BY 
                      num_leagues DESC"))

Adding the number of leagues of each player to the player table using their player_api_id.

In [12]:
Player <- Player %>% inner_join(ques_4, by = "player_api_id")

Now the Player table has the number of leagues sorted according to the player_api_id.

In [13]:
Player

Source:   query [?? x 6]
Database: sqlite 3.11.1 [Desktop/database.sqlite]

         player_name player_api_id overall_rating gk_attributes num_appearances
               <chr>         <int>          <int>         <int>           <int>
1  Patryk Rachwal,18          2625             63           153              90
2        Diego Mainz          2752             72            53              81
3        Jose Dorado          2768             75           128              71
4   Ignacio Gonzalez          2770             74           148              16
5        Alberto Rey          2790             73           144              50
6     Javier Jimenez          2796             69           340               1
7    Pablo Hernandez          2802             82           146             144
8        Ruben Perez          2805             76           141             131
9         Ivan Perez          2857             72           143               3
10   Vicente Sanchez          2862          

In [14]:
write.table(Player, file="Player.csv")

5) Which are the top 10 teams by sum of build up play attributes?

Querying the Team and Team_Attribute table to extract the sum of buildUpPlay_attributes along with the teams name and api_id.

In [15]:
Team <- tbl(my_db,sql("SELECT team_long_name, Team.team_api_id, MAX(buildUpPlaySpeed + buildUpPlayDribbling + buildUpPlayPassing) AS buildUpPlay_attributes FROM Team, Team_Attributes WHERE Team.team_api_id = Team_Attributes.team_api_id GROUP BY Team.team_api_id"))

Checking the contents of team

In [16]:
Team

Source:   query [?? x 3]
Database: sqlite 3.11.1 [Desktop/database.sqlite]

             team_long_name team_api_id buildUpPlay_attributes
                      <chr>       <int>                  <int>
1              Ruch Chorzów        1601                    139
2       Oud-Heverlee Leuven        1773                    153
3     Jagiellonia Białystok        1957                    158
4            S.C. Olhanense        2033                    147
5               Lech Poznań        2182                    176
6               P. Warszawa        2183                     NA
7                  Cracovia        2186                    125
8  Évian Thonon Gaillard FC        4087                    156
9  US Boulogne Cote D'Opale        4170                     NA
10                   Novara        6269                    179
# ... with more rows

6) Which are the top 10 teams by sum of chance creation attributes?

Querying the Team and Team_Attribute table to extract the sum of chanceCreation_attributes along with the team_api_id.

In [17]:
ques_6 <- tbl(my_db,sql("SELECT team_api_id,MAX(chanceCreationPassing + chanceCreationCrossing + chanceCreationShooting) AS chanceCreation_attributes FROM Team_Attributes GROUP BY team_api_id"))
ques_6

Source:   query [?? x 2]
Database: sqlite 3.11.1 [Desktop/database.sqlite]

   team_api_id chanceCreation_attributes
         <int>                     <int>
1         1601                       186
2         1773                       157
3         1957                       163
4         2033                       165
5         2182                       196
6         2183                       180
7         2186                       168
8         4087                       180
9         4170                       165
10        6269                       172
# ... with more rows

Adding the chanceCreation_attributes of each team to the team table using their team_api_id.

In [18]:
Team <- Team %>% left_join(ques_6)

Joining, by = "team_api_id"


Now the team table has the chanceCreation_attributes sorted according to the team_api_id.

In [19]:
Team

Source:   query [?? x 4]
Database: sqlite 3.11.1 [Desktop/database.sqlite]

             team_long_name team_api_id buildUpPlay_attributes
                      <chr>       <int>                  <int>
1              Ruch Chorzów        1601                    139
2       Oud-Heverlee Leuven        1773                    153
3     Jagiellonia Białystok        1957                    158
4            S.C. Olhanense        2033                    147
5               Lech Poznań        2182                    176
6               P. Warszawa        2183                     NA
7                  Cracovia        2186                    125
8  Évian Thonon Gaillard FC        4087                    156
9  US Boulogne Cote D'Opale        4170                     NA
10                   Novara        6269                    179
# ... with more rows, and 1 more variables: chanceCreation_attributes <int>

7) Which are the top 10 teams by sum of defense attributes?

Querying the Team and Team_Attribute table to extract the sum of defence_attributes along with the team_api_id.

In [20]:
ques_7 <- tbl(my_db,sql("SELECT team_api_id,MAX(defencePressure + defenceAggression + defenceTeamWidth) AS defence_attributes FROM Team_Attributes GROUP BY team_api_id"))
ques_7

Source:   query [?? x 2]
Database: sqlite 3.11.1 [Desktop/database.sqlite]

   team_api_id defence_attributes
         <int>              <int>
1         1601                175
2         1773                137
3         1957                210
4         2033                155
5         2182                160
6         2183                175
7         2186                205
8         4087                165
9         4170                200
10        6269                151
# ... with more rows

Adding the defense_attributes of each team to the team table using their team_api_id.

In [21]:
Team <- Team %>% left_join(ques_7)

Joining, by = "team_api_id"


Now the team table has the defense_attributes sorted according to the team_api_id.

In [22]:
Team

Source:   query [?? x 5]
Database: sqlite 3.11.1 [Desktop/database.sqlite]

             team_long_name team_api_id buildUpPlay_attributes
                      <chr>       <int>                  <int>
1              Ruch Chorzów        1601                    139
2       Oud-Heverlee Leuven        1773                    153
3     Jagiellonia Białystok        1957                    158
4            S.C. Olhanense        2033                    147
5               Lech Poznań        2182                    176
6               P. Warszawa        2183                     NA
7                  Cracovia        2186                    125
8  Évian Thonon Gaillard FC        4087                    156
9  US Boulogne Cote D'Opale        4170                     NA
10                   Novara        6269                    179
# ... with more rows, and 2 more variables: chanceCreation_attributes <int>,
#   defence_attributes <int>

9) Which are the top 10 teams by number of scored goals?

Calculating the number of goals scored by each team on home ground.

In [23]:
g1<- tbl(my_db,sql("SELECT home_team_api_id as team_api_id, sum(home_team_goal) as Home  FROM Match GROUP BY home_team_api_id"))
g1

Source:   query [?? x 2]
Database: sqlite 3.11.1 [Desktop/database.sqlite]

   team_api_id  Home
         <int> <int>
1         1601   148
2         1773    74
3         1957   176
4         2033    70
5         2182   215
6         2183    96
7         2186   144
8         4049    22
9         4064    13
10        4087   105
# ... with more rows

Calculating the number of goals scored by each team on away ground.

In [24]:
g2 <-tbl(my_db,sql("SELECT away_team_api_id as team_api_id, sum(away_team_goal) as Away FROM Match GROUP BY away_team_api_id"))
g2

Source:   query [?? x 2]
Database: sqlite 3.11.1 [Desktop/database.sqlite]

   team_api_id  Away
         <int> <int>
1         1601   132
2         1773    52
3         1957   111
4         2033    68
5         2182   157
6         2183    88
7         2186    91
8         4049    13
9         4064    14
10        4087    75
# ... with more rows

Joining the two tables by the team_api_id and adding up the rows to get the total number of goals.

In [25]:
g2 <- g1 %>% left_join(g2, by = "team_api_id")
g2<-g2 %>% mutate(goals = Home + Away)
g2

Source:   query [?? x 4]
Database: sqlite 3.11.1 [Desktop/database.sqlite]

   team_api_id  Home  Away goals
         <int> <int> <int> <int>
1         1601   148   132   280
2         1773    74    52   126
3         1957   176   111   287
4         2033    70    68   138
5         2182   215   157   372
6         2183    96    88   184
7         2186   144    91   235
8         4049    22    13    35
9         4064    13    14    27
10        4087   105    75   180
# ... with more rows

Selecting only the relevant columns.

In [26]:
ques_8 <- g2 %>% select(team_api_id,goals)
ques_8

Source:   query [?? x 2]
Database: sqlite 3.11.1 [Desktop/database.sqlite]

   team_api_id goals
         <int> <int>
1         1601   280
2         1773   126
3         1957   287
4         2033   138
5         2182   372
6         2183   184
7         2186   235
8         4049    35
9         4064    27
10        4087   180
# ... with more rows

In [27]:
Team <- Team %>% left_join(ques_8)

Joining, by = "team_api_id"


Now the team table has the number of goals by each team sorted according to the team_api_id.

In [28]:
Team

Source:   query [?? x 6]
Database: sqlite 3.11.1 [Desktop/database.sqlite]

             team_long_name team_api_id buildUpPlay_attributes
                      <chr>       <int>                  <int>
1              Ruch Chorzów        1601                    139
2       Oud-Heverlee Leuven        1773                    153
3     Jagiellonia Białystok        1957                    158
4            S.C. Olhanense        2033                    147
5               Lech Poznań        2182                    176
6               P. Warszawa        2183                     NA
7                  Cracovia        2186                    125
8  Évian Thonon Gaillard FC        4087                    156
9  US Boulogne Cote D'Opale        4170                     NA
10                   Novara        6269                    179
# ... with more rows, and 3 more variables: chanceCreation_attributes <int>,
#   defence_attributes <int>, goals <int>

In [30]:
write.table(Team, file="Team.csv")

[1] http://stackoverflow.com/questions/14789928/count-occurrences-of-multiple-columns-as-total